# Common以下のコード補足

In [28]:
import numpy as np

***
# functions

## softmax関数
xが1次元，2次元の場合に分けて，オーバーフロー対策のために各データから最大値を引く

In [29]:
x = np.random.rand(3, 10)
print(x)
print(x.max(axis=1, keepdims=True))
print(x - x.max(axis=1, keepdims=True))

[[0.17272812 0.70646724 0.59279828 0.78452282 0.07242614 0.27018514
  0.52959084 0.44054175 0.29941408 0.95227623]
 [0.84816193 0.82349508 0.26071452 0.08811669 0.78277514 0.0873502
  0.31550918 0.64270237 0.98627119 0.77822009]
 [0.84659366 0.64145101 0.47245951 0.2832024  0.4686912  0.90214795
  0.3844346  0.42023613 0.37207814 0.0291764 ]]
[[0.95227623]
 [0.98627119]
 [0.90214795]]
[[-0.77954812 -0.245809   -0.35947796 -0.16775341 -0.87985009 -0.68209109
  -0.4226854  -0.51173448 -0.65286215  0.        ]
 [-0.13810926 -0.1627761  -0.72555667 -0.8981545  -0.20349605 -0.89892099
  -0.67076201 -0.34356881  0.         -0.2080511 ]
 [-0.05555429 -0.26069694 -0.42968844 -0.61894555 -0.43345675  0.
  -0.51771335 -0.48191182 -0.53006981 -0.87297155]]


## cross_entropy_error
- 1データ分
$$
L = - \Sigma_k t_k \log y_k
$$

- バッチ
$$
L = - \frac{1}{N} \Sigma_n \Sigma_k t_{nk} \log y_{nk}
$$

- targetラベルがone-hot-vectorであれば，正解ラベルに対応するlog yを計算するだけ．

In [30]:
# 教師データのone-hot-vectorを正解ラベルのインデックスに変換
t = np.array([[0, 0, 1, 0],
              [1, 0, 0, 0],
              [0, 0, 0, 1]])  # target
y = np.random.rand(*t.shape)  # 入力
print(f'y_org: \n {y}')
print(f't_org: \n {t}')

batch_size = y.shape[0]  # バッチ

t = t.argmax(axis=1)  # targetをone-hot-vectorからインデックスに変換

out = -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size
print(f'out: \n {out}')
print(f't_argmax: \n {t}')

y_org: 
 [[0.11492679 0.36854857 0.59007722 0.24492445]
 [0.91455799 0.20699962 0.14024878 0.32328387]
 [0.66044397 0.61200888 0.43284912 0.76594419]]
t_org: 
 [[0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]]
out: 
 0.294487273966712
t_argmax: 
 [2 0 3]


***
# layers

## SigmoidWithLoss
$$
y = \frac{1}{1 + \exp(-x)}
\\ \\
L = - \{ t \log y + (1 - t) \log(1 - y) \}
$$
- CBOWモデルの高速化(多値分類を二値分類に置き換える)で使用．2クラス分類．
- 答えが不正解のときt = 0, 正解のときt = 1が渡される．

In [31]:
x = np.random.rand(5).reshape(5, 1)  # 入力
y = 1 / (1 + np.exp(-x))  # Sigmoid
# 以下の配列と t = 0 or 1 のラベルをcross_entropy_errorに渡せばよい
print(y)
print(np.c_[1 - y, y])

[[0.52394016]
 [0.61057432]
 [0.72651129]
 [0.50986536]
 [0.67894195]]
[[0.47605984 0.52394016]
 [0.38942568 0.61057432]
 [0.27348871 0.72651129]
 [0.49013464 0.50986536]
 [0.32105805 0.67894195]]


## Embedding
- CBOWモデルの高速化．入力層->中間層の全結合層MatMulレイヤの代わりに使う．
- MuｔMulでやったことは単語IDのone-hot-vectorと重みW_inの積和で，これはW_inから単語IDに対応するインデックスの行を抜き出すことに等しい

In [32]:
# forwardメソッド
W = np.random.rand(5, 3)  # 重み
contexts = np.array([[0, 2],
                     [1, 3],
                     [2, 4],
                     [3, 0],
                     [4, 2]])  # window_size=2, n=6のコンテキスト

print(f'W: \n {W}')

# コンテキストの数と同じだけEmbeddingレイヤを使ってコンテキストの列ごとに処理する
for i in range(contexts.shape[1]) :
    idx = contexts[:, i]
    out = W[idx]
    print(f'out_{i}: \n {out}')

W: 
 [[0.03946521 0.40274705 0.7892266 ]
 [0.58376072 0.44757264 0.53699497]
 [0.5032567  0.18050419 0.52080341]
 [0.0785154  0.53291386 0.14954686]
 [0.9767782  0.05266102 0.0872479 ]]
out_0: 
 [[0.03946521 0.40274705 0.7892266 ]
 [0.58376072 0.44757264 0.53699497]
 [0.5032567  0.18050419 0.52080341]
 [0.0785154  0.53291386 0.14954686]
 [0.9767782  0.05266102 0.0872479 ]]
out_1: 
 [[0.5032567  0.18050419 0.52080341]
 [0.0785154  0.53291386 0.14954686]
 [0.9767782  0.05266102 0.0872479 ]
 [0.03946521 0.40274705 0.7892266 ]
 [0.5032567  0.18050419 0.52080341]]


In [33]:
# backwardメソッド
grad = np.ones_like(W)  # 勾配
dW = grad
dW[...] = 0  # 勾配を初期化

dout = np.arange(W.size).reshape(W.shape)
print(f'dW_init: \n {dW}')
print(f'dout: \n {dout}')
print(f'idx: \n {contexts[:, 1]}')

np.add.at(dW, contexts[:, 1], dout)  # dWの指定した行にdoutの各行を足していく
print(f'dW: \n {dW}')

dW_init: 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
dout: 
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]]
idx: 
 [2 3 4 0 2]
dW: 
 [[ 9. 10. 11.]
 [ 0.  0.  0.]
 [12. 14. 16.]
 [ 3.  4.  5.]
 [ 6.  7.  8.]]


***
# trainer
## Trainer

In [34]:
# データのシャッフル
np.random.permutation(10)

array([3, 7, 6, 9, 2, 5, 4, 8, 1, 0])

In [35]:
x = np.random.randn(1000)
data_size = len(x)
batch_size = 32
max_iters = data_size // batch_size  # 31
max_epoch = 10
eval_interval = 20

In [36]:
# fitメソッド
for epoch in range(max_epoch):
    print(f'--------------epoch: {epoch}--------------')
    print('ここでデータをシャッフル')
    
    for iters in range(max_iters):
        print(f'--------iters: {iters}--------')
        print('ミニバッチを取得')
        print('順伝播で損失計算')
        print('逆伝播で勾配計算')
        print('max_gradを指定していれば勾配クリッピング')
        print('パラメータ更新 (optimizer.update)')
        print('total_lossに損失の値を足し上げる')
        print('loss_countに損失関数の計算回数をカウント')
        
        if (eval_interval is not None) and (iters % eval_interval) == 0:
            print('------eval------')
            print('損失の平均を計算，出力，リスト追加')
            print('total_loss, loss_countを初期化')

--------------epoch: 0--------------
ここでデータをシャッフル
--------iters: 0--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
------eval------
損失の平均を計算，出力，リスト追加
total_loss, loss_countを初期化
--------iters: 1--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 2--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 3--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 4--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 5--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
lo

パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 7--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 8--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 9--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 10--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 11--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 12--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------it

--------iters: 17--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 18--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 19--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 20--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
------eval------
損失の平均を計算，出力，リスト追加
total_loss, loss_countを初期化
--------iters: 21--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 22--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 23---

パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 16--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 17--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 18--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 19--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 20--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
------eval------
損失の平均を計算，出力，リスト追加
total_loss, loss_countを初期化
--------iters: 21--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.u

In [94]:
# remove_duplicate関数
param0 = np.random.rand(3, 2)
param1 = np.random.rand(3, 2)
param2 = param1

grad0 = np.random.rand(*param1.shape)
grad1 = np.random.rand(*param2.shape)
grad2 = np.random.rand(*param3.shape)

params = [param0, param1, param2]
grads = [grad0, grad1, grad2]

for param in params:
    print(param)

print('')

for grad in grads:
    print(grad)

[[0.41329701 0.27429692]
 [0.43928461 0.67687584]
 [0.66102804 0.57758767]]
[[0.40852099 0.9599229 ]
 [0.38798494 0.14466335]
 [0.78084202 0.91797246]]
[[0.40852099 0.9599229 ]
 [0.38798494 0.14466335]
 [0.78084202 0.91797246]]

[[0.26322454 0.79547303]
 [0.79299877 0.0171328 ]
 [0.54268622 0.36540175]]
[[0.74186737 0.91525073]
 [0.72521673 0.88973107]
 [0.63998387 0.47083918]]
[[0.00828632 0.34767151]
 [0.56278861 0.98147675]
 [0.60633485 0.73105223]]


In [95]:
def remove_duplicate(params, grads):
    params, grads = params[:], grads[:]  # copy list

    while True:
        find_flg = False
        L = len(params)

        for i in range(0, L - 1):
            for j in range(i + 1, L):
                # 重みを共有する場合
                if params[i] is params[j]:
                    grads[i] += grads[j]  # 勾配の加算
                    find_flg = True
                    params.pop(j)
                    grads.pop(j)
                # 転置行列として重みを共有する場合（weight tying）
                elif params[i].ndim == 2 and params[j].ndim == 2 and \
                     params[i].T.shape == params[j].shape and np.all(params[i].T == params[j]):
                    grads[i] += grads[j].T
                    find_flg = True
                    params.pop(j)
                    grads.pop(j)

                if find_flg: break
            if find_flg: break

        if not find_flg: break

    return params, grads

params_removed, grads_removed = remove_duplicate(params, grads)

In [96]:
for param in params_removed:
    print(param)

print('')

for grad in grads_removed:
    print(grad)

[[0.41329701 0.27429692]
 [0.43928461 0.67687584]
 [0.66102804 0.57758767]]
[[0.40852099 0.9599229 ]
 [0.38798494 0.14466335]
 [0.78084202 0.91797246]]

[[0.26322454 0.79547303]
 [0.79299877 0.0171328 ]
 [0.54268622 0.36540175]]
[[0.7501537  1.26292224]
 [1.28800534 1.87120782]
 [1.24631872 1.20189141]]


In [82]:
params[0] is params[2]

False